# Azure 완료 예제
이 예제에서는 Azure 엔드포인트를 사용하여 완료를 작동시키는 데 필요한 모든 작업을 살펴보겠습니다. \
이 예에서는 완료에 중점을 두지만 API를 사용하여 사용할 수 있는 몇 가지 다른 작업도 다룹니다. 이 예제는 간단한 연산을 빠르게 보여주기 위한 것이며 자습서가 아닙니다.

In [ ]:
import os
import openai

## 설정
다음 섹션이 제대로 작동하려면 먼저 몇 가지를 설정해야 합니다. 먼저 `api_base`와 `api_version`부터 설정해 보겠습니다. https://portal.azure.com 로 이동하여 리소스를 찾은 다음 "리소스 관리" -> "키 및 엔드포인트"에서 "엔드포인트" 값을 찾습니다.

In [ ]:
openai.api_version = '2023-05-15'
openai.api_base = '' # Please add your endpoint here

다음으로 `api_type`과 `api_key`를 설정해야 합니다. 포털에서 키를 가져오거나 Microsoft Active Directory 인증을 통해 키를 가져올 수 있습니다. 이에 따라 `api_type`은 `azure` 또는 `azure_ad`입니다.

### 설정: 포털
먼저 포털에서 키를 가져오는 방법을 살펴봅시다. https://portal.azure.com 로 이동하여 리소스를 찾은 다음 "리소스 관리" -> "키 및 엔드포인트"에서 "키" 값 중 하나를 찾습니다.

In [ ]:
openai.api_type = 'azure'
openai.api_key = os.environ["OPENAI_API_KEY"]

> 참고: 이 예제에서는 코드에서 변수를 설정하여 Azure API를 사용하도록 라이브러리를 구성했습니다. 개발의 경우 환경 변수를 대신 설정하는 것을 고려하세요:

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

### (선택 사항) 설정: Microsoft Active Directory 인증
이제 Microsoft Active Directory 인증을 통해 키를 얻는 방법을 살펴보겠습니다. 포털의 키 대신 Active Directory 인증을 사용하려면 다음 코드의 주석 처리를 해제하세요.

In [ ]:
# from azure.identity import DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token

토큰은 일정 기간 동안 유효하며 그 이후에는 만료됩니다. 모든 요청에 유효한 토큰이 전송되도록 하려면 요청.auth에 연결하여 만료되는 토큰을 새로 고칠 수 있습니다:

In [ ]:
import typing
import time
import requests
if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

session = requests.Session()
session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

openai.requestssession = session

## 배포
이 섹션에서는 `text-davinci-002` 모델을 사용하여 배포를 생성한 다음 완료를 생성하는 데 사용할 수 있습니다.

### 배포: 수동으로 생성하기
포털의 리소스로 이동하여 "리소스 관리" -> "모델 배포"에서 새 배포를 만듭니다. 모델로 `text-davinci-002`를 선택합니다.

In [ ]:
deployment_id = '' # Fill in the deployment id from the portal here

### 완료
이제 샘플 완료를 배포로 전송해 보겠습니다.

In [ ]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id=deployment_id,
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")